In [35]:
import json
import requests
import urllib.parse
import io
import matplotlib.pyplot as plt
from astropy.table import Table
import os
import re
import sys
import time
import matplotlib
import pandas as pd

# VO TAP queries via astroquery:
from astroquery.utils.tap.core import TapPlus

# Additional information see: https://ned.ipac.caltech.edu
# TEST DATA: https://gcn.gsfc.nasa.gov/notices_amon_g_b/139204_39158985.amon
output_dir = r'C:\Users\jhzhe\Downloads'
ned_tab = "https://ned.ipac.caltech.edu/tap/sync"
table_name = 'objdir'
columns = 'RA,Dec'
coord_sys = 'J2000'
coord_ra = 7.7106  # From previous example - ned_object_basic_info['Preferred']['Position']['RA']
coord_dec = 30.80  # From previous example - ned_object_basic_info['Preferred']['Position']['Dec']
sr_deg = 18.59 / 60  # 0.2 arcmin search radius
cone = f"CONTAINS(POINT('{coord_sys}', RA, Dec), CIRCLE('{coord_sys}', {coord_ra}, {coord_dec}, {sr_deg}))=1"
query = f"SELECT TOP 40 {columns} FROM {table_name} WHERE {cone}"

# Using astroquery
ned = TapPlus(url=ned_tab)
job = ned.launch_job(query)
out = job.get_results()
coord_data = out.to_pandas()
output_path = os.path.join(output_dir, 'NED_TEST_3.csv')


## Connect to ATLAS API, to fetch data from csv

In [33]:
# Define the input/output directory
input_dir_txt = r'c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data'
input_dir_csv = r'c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data_csv'
output_dir = r'c:/Users/jhzhe/Documents/MPhys_Neutrino/output_data/test_data'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)


BASEURL = "https://fallingstar-data.com/forcedphot"
# BASEURL = "http://127.0.0.1:8000"

if os.environ.get("ATLASFORCED_SECRET_KEY"):
    token = os.environ.get("ATLASFORCED_SECRET_KEY")
    print("Using stored token")
else:
    data = {"username": "JinZheng", "password": "NB,E7,SNCL5mx"}

    resp = requests.post(url=f"{BASEURL}/api-token-auth/", data=data)

    if resp.status_code == 200:
        token = resp.json()["token"]
        print(f"Your token is {token}")
        print("Store this by running/adding to your .zshrc file:")
        print(f'export ATLASFORCED_SECRET_KEY="{token}"')
    else:
        print(f"ERROR {resp.status_code}")
        print(resp.text)
        sys.exit()


headers = {"Authorization": f"Token {token}", "Accept": "application/json"}


def fetch_data(ra, dec, mjd_min=58582.8, mjd_max=58782.8, index=1):
    task_url = None
    while not task_url:
        with requests.Session() as s:
            # alternative to token auth
            # s.auth = ('USERNAME', 'PASSWORD')
            resp = s.post(f"{BASEURL}/queue/", headers=headers, data={"ra": ra, "dec": dec, "mjd_min": mjd_min,
                                                                      "mjd_max": mjd_max})

            if resp.status_code == 201:  # successfully queued
                task_url = resp.json()["url"]
                print(f"The task URL is {task_url}")
            elif resp.status_code == 429:  # throttled
                message = resp.json()["detail"]
                print(f"{resp.status_code} {message}")
                t_sec = re.findall(r"available in (\d+) seconds", message)
                t_min = re.findall(r"available in (\d+) minutes", message)
                if t_sec:
                    waittime = int(t_sec[0])
                elif t_min:
                    waittime = int(t_min[0]) * 60
                else:
                    waittime = 10
                print(f"Waiting {waittime} seconds")
                time.sleep(waittime)
            else:
                print(f"ERROR {resp.status_code}")
                print(resp.text)
                sys.exit()


    result_url = None
    taskstarted_printed = False
    while not result_url:
        with requests.Session() as s:
            resp = s.get(task_url, headers=headers)

            if resp.status_code == 200:  # HTTP OK
                if resp.json()["finishtimestamp"]:
                    result_url = resp.json()["result_url"]
                    print(f"Task is complete with results available at {result_url}")
                elif resp.json()["starttimestamp"]:
                    if not taskstarted_printed:
                        print(f"Task is running (started at {resp.json()['starttimestamp']})")
                        taskstarted_printed = True
                    time.sleep(2)
                else:
                    print(f"Waiting for job to start (queued at {resp.json()['timestamp']})")
                    time.sleep(4)
            else:
                print(f"ERROR {resp.status_code}")
                print(resp.text)
                sys.exit()

    with requests.Session() as s:
        textdata = s.get(result_url, headers=headers).text
        file_path_data_txt = os.path.join(input_dir_txt, f'text_data{index}.txt')
        with open(file_path_data_txt, 'w') as f:
            f.write(textdata)
        print(f"Raw text data saved to {file_path_data_txt}")
        # if we'll be making a lot of requests, keep the web queue from being
        # cluttered (and reduce server storage usage) by sending a delete operation
        # s.delete(task_url, headers=headers).json()

    dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)

    file_path_data = os.path.join(input_dir_csv, f'data{index}.csv')
    
    dfresult.to_csv(file_path_data)


Your token is 938191f3499bfce3ac5b3566e298b2c9106d84d3
Store this by running/adding to your .zshrc file:
export ATLASFORCED_SECRET_KEY="938191f3499bfce3ac5b3566e298b2c9106d84d3"


In [39]:
ra_list = coord_data['ra']
dec_list = coord_data['dec']
mjd_min = 20396 + 40000
mjd_max = 20596 + 40000

for a, (b,c) in enumerate(zip(ra_list, dec_list)):
    fetch_data(b, c, mjd_min=mjd_min, mjd_max=mjd_max, index = a)


The task URL is https://fallingstar-data.com/forcedphot/queue/1824474/
Task is running (started at 2024-11-04T18:32:37Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1824474.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data0.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1824479/
Task is running (started at 2024-11-04T18:35:11Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1824479.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data1.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1824480/
Task is running (started at 2024-11-04T18:35:35Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1824480.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data2.txt


...

The task URL is https://fallingstar-data.com/forcedphot/queue/1824548/
Task is running (started at 2024-11-04T19:02:37Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1824548.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data21.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1824553/
Task is running (started at 2024-11-04T19:07:31Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1824553.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data22.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [41]:
for a, (b,c) in enumerate(zip(ra_list[23:], dec_list[23:]), start = 23):
    fetch_data(b, c, mjd_min=mjd_min, mjd_max=mjd_max, index = a)

The task URL is https://fallingstar-data.com/forcedphot/queue/1827103/
Task is running (started at 2024-11-05T15:47:39Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1827103.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data23.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1827104/
Task is running (started at 2024-11-05T15:48:06Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1827104.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data24.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1827106/
Task is running (started at 2024-11-05T15:50:07Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1827106.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data25.txt


...

C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1827124/
Task is running (started at 2024-11-05T16:02:08Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1827124.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data38.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1827128/
Task is running (started at 2024-11-05T16:04:11Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1827128.txt
Raw text data saved to c:/Users/jhzhe/Documents/MPhys_Neutrino/input_data/test_data\text_data39.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_15820\227255094.py:99: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


In [44]:
from plot_atlas_fp import plotter
import logging
# Initiliasing the plotter class
resultsFilePath = [os.path.join(input_dir_txt, file) for file in os.listdir(input_dir_txt) if file.endswith('.txt')]

logging.basicConfig(level=logging.INFO) 
log = logging.getLogger(__name__)
myPlotter = plotter(
    log = log,
    resultFilePaths = resultsFilePath,
    outputDirectory = output_dir,
    plotType = "png",
    RA_list = ra_list,
    dec_list = dec_list
)

plotPaths = myPlotter.plot_compile()

INFO:__main__:starting the ``plot_compile`` method
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_c``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_o``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_c``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_o``
INFO:__main__:starting the ``plot_single_result_compile_o`` method
INFO:__main__:starting the ``read_and_sigma_clip_data`` function
INFO:__main__:completed ``plot_single_result_compile_c``
INFO

In [45]:
plotPaths

['C:\\Users\\jhzhe\\Downloads//combined_plot_c.png',
 'C:\\Users\\jhzhe\\Downloads//combined_plot_o.png']